# 코사인 유사도를 이용한 영화 추천 시스템

## 코사인 유사도

In [1]:
from numpy import dot           # 점표현
from numpy.linalg import norm   # 벡터의 크기
import numpy as np
def cos_sim(A, B):
       return dot(A, B)/(norm(A)*norm(B))

In [2]:
doc1=np.array([0,1,1,1])
doc2=np.array([1,0,1,1])
doc3=np.array([2,0,2,2])

In [3]:
cos_sim(doc1, doc2), cos_sim(doc1, doc3), cos_sim(doc2, doc3)

(0.6666666666666667, 0.6666666666666667, 1.0000000000000002)

In [4]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity([doc1], [doc2])

array([[0.66666667]])

In [5]:
cosine_similarity([doc3], [doc2])

array([[1.]])

## 유사도를 이용한 추천 시스템

In [8]:
from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving movies_metadata.csv to movies_metadata (3).csv


In [9]:
import pandas as pd

movie = pd.read_csv(filename, low_memory=False)
movie.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [10]:
movie.shape

(45466, 24)

In [11]:
df = movie[['title', 'overview']]
df.head(2)

,title,overview
0,Toy Story,"Led by Woody, Andy's toys live happily in his ..."
1,Jumanji,When siblings Judy and Peter discover an encha...


In [12]:
df.overview[0]

"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences."

In [13]:
df = df.head(20000)

## 데이터 전처리

In [14]:
df.isnull().sum()

title         2
overview    135
dtype: int64

In [15]:
import warnings
warnings.filterwarnings('ignore')

In [16]:
df.dropna(inplace=True)   # how='any' 가 기본값이므로 생략

In [17]:
df.isnull().sum()

title       0
overview    0
dtype: int64

In [18]:
df.shape

(19863, 2)

In [19]:
# data는 삭제되었지만 남은 데이터의 index 번호는 그대로 남음
df.tail()

,title,overview
19995,Rebellion,Dissidents in a French colony attack a police ...
19996,Versailles,A young mother Nina and her son Enzo find them...
19997,Two in the Wave,An in-depth analysis of the relationship betwe...
19998,Lotte Reiniger: Homage to the Inventor of the ...,Follows the life and work of animator Lotte Re...
19999,"RKO Production 601: The Making of 'Kong, the E...","An in-depth look at the genesis, production, a..."


In [20]:
df.set_index('title', inplace=True)
df.reset_index(inplace=True)
df.tail()

,title,overview
19858,Rebellion,Dissidents in a French colony attack a police ...
19859,Versailles,A young mother Nina and her son Enzo find them...
19860,Two in the Wave,An in-depth analysis of the relationship betwe...
19861,Lotte Reiniger: Homage to the Inventor of the ...,Follows the life and work of animator Lotte Re...
19862,"RKO Production 601: The Making of 'Kong, the E...","An in-depth look at the genesis, production, a..."


## 텍스트 전처리

In [21]:
# 구둣점 제거
import re
# df['clean_doc'] = df.overview.apply(lambda x: re.sub('[^A-Za-z ]', '', x))
df['clean_doc'] = df.overview.str.replace('[^A-Za-z ]', '')

## DTM 변환

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
tvect = TfidfVectorizer(stop_words='english')
tfidf_matrix = tvect.fit_transform(df.overview)
tfidf_matrix.shape

(19863, 47487)

In [23]:
tfidf_clean = tvect.fit_transform(df.clean_doc)
tfidf_clean.shape

(19863, 54245)

### 영화의 타이틀과 인덱스를 가진 테이블

In [24]:
indices = pd.Series(df.index, index=df.title).drop_duplicates()
indices.head()

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
dtype: int64

In [25]:
indices['Jumanji']

1

## 코사인 유사도를 통해 유사한 영화를 찾는 함수

In [26]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_clean = linear_kernel(tfidf_clean, tfidf_clean)

In [27]:
cosine_sim.shape, cosine_clean.shape

((19863, 19863), (19863, 19863))

In [29]:
cosine_sim[1, :5]

array([0.01575156, 1.        , 0.04906868, 0.        , 0.        ])

In [30]:
cosine_clean[1, :5]

array([0.01538829, 1.        , 0.02638578, 0.        , 0.        ])

In [31]:
def get_recommendations(title, cosine_sim=cosine_sim):
    # 선택한 영화의 타이틀로부터 해당되는 인덱스를 받아옵니다. 이제 선택한 영화를 가지고 연산할 수 있습니다.
    idx = indices[title]

    # 모든 영화에 대해서 해당 영화와의 유사도를 구합니다.
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 영화들을 정렬합니다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 영화를 받아옵니다.
    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개의 영화의 인덱스를 받아옵니다.
    movie_indices = [i[0] for i in sim_scores]

    # 가장 유사한 10개의 영화의 제목을 리턴합니다.
    return df.title.iloc[movie_indices]

In [33]:
def get_rec_clean(title, cosine_sim=cosine_clean):
    # 선택한 영화의 타이틀로부터 해당되는 인덱스를 받아옵니다. 이제 선택한 영화를 가지고 연산할 수 있습니다.
    idx = indices[title]

    # 모든 영화에 대해서 해당 영화와의 유사도를 구합니다.
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 영화들을 정렬합니다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 영화를 받아옵니다.
    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개의 영화의 인덱스를 받아옵니다.
    movie_indices = [i[0] for i in sim_scores]

    # 가장 유사한 10개의 영화의 제목을 리턴합니다.
    return df.title.iloc[movie_indices]

In [32]:
get_recommendations('The Dark Knight Rises')

12447                            The Dark Knight
149                               Batman Forever
1314                              Batman Returns
15444                 Batman: Under the Red Hood
583                                       Batman
9203          Batman Beyond: Return of the Joker
17930                           Batman: Year One
19661    Batman: The Dark Knight Returns, Part 1
3077                Batman: Mask of the Phantasm
10092                              Batman Begins
Name: title, dtype: object

In [34]:
get_rec_clean('The Dark Knight Rises')

1314                              Batman Returns
12447                            The Dark Knight
15444                 Batman: Under the Red Hood
583                                       Batman
149                               Batman Forever
19661    Batman: The Dark Knight Returns, Part 1
3077                Batman: Mask of the Phantasm
9203          Batman Beyond: Return of the Joker
17930                           Batman: Year One
10092                              Batman Begins
Name: title, dtype: object